# Training an SNN using surrogate gradients!

This notebook contains the NMNIST results for arctan surrogate gradient

In [1]:
import jax.numpy as jnp

warmup = jnp.array([12,34])
warmup * 12.34

Array([148.08, 419.56], dtype=float32, weak_type=True)

In [2]:
# implement our SNN in DeepMind's Haiku
import haiku as hk

# JAX imports
import jax
import jmp

# rendering tools
# for surrogate loss training.
import optax
from jax import numpy as jnp
from jax_tqdm import scan_tqdm

import spyx
import spyx.nn as snn

%matplotlib notebook

## Set Mixed Precision Policy

In [3]:
policy = jmp.get_policy('half')


hk.mixed_precision.set_policy(hk.Linear, policy)
hk.mixed_precision.set_policy(snn.LIF, policy)
hk.mixed_precision.set_policy(snn.LI, policy)

## Data Loading

In [4]:
nmnist_dl = spyx.data.NMNIST_loader(64)

In [5]:
key = jax.random.PRNGKey(0)
x, y = nmnist_dl.train_epoch(key)

In [18]:
x.shape

(216, 64, 5, 2, 34, 34)

In [6]:
y.shape

(216, 64)

## SNN

Here we define a simple feed-forward SNN using Haiku's RNN features, incorporating our
LIF neuron models where activation functions would usually go. Haiku manages all of the state for us, so when we transform the function and get an apply() function we just need to pass the params!

Since spiking neurons have a discrete all-or-nothing activation, in order to do gradient descent we'll have to approximate the derivative of the Heaviside function with something smoother. In this case, we use the SuperSpike surrogate gradient from Zenke & Ganguli 2017.
Also not that we aren't using bias terms on the linear layers and since the inputs are images, we flatten the data before feeding it to the first layer.

Depending on computational constraints, we can use haiku's dynamic unroll to iterate the SNN, or we can use static unroll where the SNN will be unrolled during the JIT compiling process to further increase speed when training on GPU. Note that the static unroll will take longer to compile, but once it runs the iterations per second will be 2x-3x greater than the dynamic unroll.

In [7]:
surrogate = spyx.axn.Axon(spyx.axn.arctan())

def nmnist_snn(x):    
    # seqs is [T, F].
    
    x = hk.BatchApply(hk.Flatten())(x)
    x = hk.BatchApply(hk.Linear(512, with_bias=False))(x)
    
    core = hk.DeepRNN([
        snn.LIF((512,), activation=surrogate),
        hk.Linear(10, with_bias=False),
        snn.LI((10,))
    ])
    
    # static unroll for maximum performance
    spikes, V = hk.dynamic_unroll(core, x, core.initial_state(x.shape[0]), time_major=False, unroll=20)
    
    return spikes, V


## Gradient Descent

We define a training loop below.

We use the Lion optimizer from Optax, which is a more efficient competitor to the popular Adam. The eval steps and updates are JIT'ed to maximize time spent in optimized GPU code and minimize time spent in higher-level python.

The use of regularizers in the spiking network will be covered in a seperate tutorial.

In [8]:
def gd(SNN, params, dl, epochs=300, schedule = 2e-4):
    
    #aug = spyx.data.shift_augment(max_shift=3, axes=(-2,-1)) # need to make this stateless
    
    opt = optax.chain(
        optax.centralize(),
        optax.lion(learning_rate=schedule)
    )
    # create and initialize the optimizer
    opt_state = opt.init(params)
    grad_params = params
        
    # define and compile our eval function that computes the loss for our SNN
    @jax.jit
    def net_eval(weights, events, targets):
        readout = SNN.apply(weights, events)
        traces, V_f = readout
        return spyx.fn.integral_crossentropy(traces, targets, smoothing=0.2)
        
    # Use JAX to create a function that calculates the loss and the gradient!
    surrogate_grad = jax.value_and_grad(net_eval) 
        
    rng = jax.random.PRNGKey(0)        
    
    # compile the meat of our training loop for speed
    @jax.jit
    def train_step(state, data):
        grad_params, opt_state = state
        events, targets = data # fix this
        events = jnp.unpackbits(events, axis=1) # decompress temporal axis
        # compute loss and gradient                    # need better augment rng
        loss, grads = surrogate_grad(grad_params, events, targets)
        # generate updates based on the gradients and optimizer
        updates, opt_state = opt.update(grads, opt_state, grad_params)
        # return the updated parameters
        new_state = [optax.apply_updates(grad_params, updates), opt_state]
        return new_state, loss
    
    # For validation epochs, do the same as before but compute the
    # accuracy, predictions and losses (no gradients needed)
    @jax.jit
    def eval_step(grad_params, data):
        events, targets = data # fix
        events = jnp.unpackbits(events, axis=1)
        readout = SNN.apply(grad_params, events)
        traces, V_f = readout
        acc, pred = spyx.fn.integral_accuracy(traces, targets)
        loss = spyx.fn.integral_crossentropy(traces, targets)
        return grad_params, jnp.array([acc, loss])
        
    
    val_data = dl.val_epoch()
    
    # Here's the start of our training loop!
    @scan_tqdm(epochs)
    def epoch(epoch_state, epoch_num):
        curr_params, curr_opt_state = epoch_state
        
        shuffle_rng = jax.random.fold_in(rng, epoch_num)
        train_data = dl.train_epoch(shuffle_rng)
        
        # train epoch
        end_state, train_loss = jax.lax.scan(
            train_step,# func
            [curr_params, curr_opt_state],# init
            train_data,# xs
            train_data.obs.shape[0]# len
        )
        
        new_params, _ = end_state
            
        # val epoch
        _, val_metrics = jax.lax.scan(
            eval_step,# func
            new_params,# init
            val_data,# xs
            val_data.obs.shape[0]# len
        )

        
        return end_state, jnp.concatenate([jnp.expand_dims(jnp.mean(train_loss),0), jnp.mean(val_metrics, axis=0)])
    # end epoch
    
    # epoch loop
    final_state, metrics = jax.lax.scan(
        epoch,
        [grad_params, opt_state], # metric arrays
        jnp.arange(epochs), # 
        epochs # len of loop
    )
    
    final_params, _ = final_state
    
                
    # return our final, optimized network.       
    return final_params, metrics

In [9]:
def test_gd(SNN, params, dl):

    @jax.jit
    def test_step(params, data):
        events, targets = data
        events = jnp.unpackbits(events, axis=1)
        readout = SNN.apply(params, events)
        traces, V_f = readout
        acc, pred = spyx.fn.integral_accuracy(traces, targets)
        loss = spyx.fn.integral_crossentropy(traces, targets)
        return params, [acc, loss, pred, targets]
    
    test_data = dl.test_epoch()
    
    _, test_metrics = jax.lax.scan(
            test_step,# func
            params,# init
            test_data,# xs
            test_data.obs.shape[0]# len
    )
    
    acc = jnp.mean(test_metrics[0])
    loss = jnp.mean(test_metrics[1])
    preds = jnp.array(test_metrics[2]).flatten()
    tgts = jnp.array(test_metrics[3]).flatten()
    return acc, loss, preds, tgts

## Training Time

We'll train the network for 50 epochs since SHD is more difficult than MNIST.

The SHD dataloader for Spyx has built-in leave-one-group-out cross validation. This is becuase the test set for SHD has two unseen speakers, so when we train our model we need to make it robust to speakers it isn't training on in the hopes of improving generalization accuracy.

In [10]:
from time import time

In [11]:
key = jax.random.PRNGKey(0)
# Since there's nothing stochastic about the network, we can avoid using an RNG as a param!
SNN = hk.without_apply_rng(hk.transform(nmnist_snn))
params = SNN.init(rng=key, x=x[0])
start = time()
grad_params, metrics = gd(SNN, params, nmnist_dl, epochs=25, schedule=2e-4)
elapsed = time() - start
print("Performance: train_loss={}, val_acc={}, val_loss={}".format(*metrics[-1]))
print("Time Elapsed:", elapsed)
acc, loss, preds, tgts = test_gd(SNN, grad_params, nmnist_dl)
print("Accuracy:", acc, "Loss:", loss)


  0%|          | 0/25 [00:00<?, ?it/s]

Performance: train_loss=0.901228666305542, val_acc=0.963994562625885, val_loss=1.3109228610992432
Time Elapsed: 12.993635416030884
Accuracy: 0.9629407 Loss: 1.3118557


In [12]:
key = jax.random.PRNGKey(7)
# Since there's nothing stochastic about the network, we can avoid using an RNG as a param!
SNN = hk.without_apply_rng(hk.transform(nmnist_snn))
params = SNN.init(rng=key, x=x[0])
start = time()
grad_params, metrics = gd(SNN, params, nmnist_dl, epochs=25, schedule=2e-4)
elapsed = time() - start
print("Performance: train_loss={}, val_acc={}, val_loss={}".format(*metrics[-1]))
print("Time Elapsed:", elapsed)
acc, loss, preds, tgts = test_gd(SNN, grad_params, nmnist_dl)
print("Accuracy:", acc, "Loss:", loss)


  0%|          | 0/25 [00:00<?, ?it/s]

Performance: train_loss=0.9087139964103699, val_acc=0.9590693116188049, val_loss=1.331193447113037
Time Elapsed: 10.810823678970337
Accuracy: 0.9602364 Loss: 1.326659


In [13]:
key = jax.random.PRNGKey(42)
# Since there's nothing stochastic about the network, we can avoid using an RNG as a param!
SNN = hk.without_apply_rng(hk.transform(nmnist_snn))
params = SNN.init(rng=key, x=x[0])
start = time()
grad_params, metrics = gd(SNN, params, nmnist_dl, epochs=25, schedule=2e-4)
elapsed = time() - start
print("Performance: train_loss={}, val_acc={}, val_loss={}".format(*metrics[-1]))
print("Time Elapsed:", elapsed)
acc, loss, preds, tgts = test_gd(SNN, grad_params, nmnist_dl)
print("Accuracy:", acc, "Loss:", loss)


  0%|          | 0/25 [00:00<?, ?it/s]

Performance: train_loss=0.9158933758735657, val_acc=0.956182062625885, val_loss=1.3441404104232788
Time Elapsed: 10.236445426940918
Accuracy: 0.96063703 Loss: 1.3356191


In [14]:
key = jax.random.PRNGKey(12345)
# Since there's nothing stochastic about the network, we can avoid using an RNG as a param!
SNN = hk.without_apply_rng(hk.transform(nmnist_snn))
params = SNN.init(rng=key, x=x[0])
start = time()
grad_params, metrics = gd(SNN, params, nmnist_dl, epochs=25, schedule=2e-4)
elapsed = time() - start
print("Performance: train_loss={}, val_acc={}, val_loss={}".format(*metrics[-1]))
print("Time Elapsed:", elapsed)
acc, loss, preds, tgts = test_gd(SNN, grad_params, nmnist_dl)
print("Accuracy:", acc, "Loss:", loss)


  0%|          | 0/25 [00:00<?, ?it/s]

Performance: train_loss=0.9062016606330872, val_acc=0.962296187877655, val_loss=1.3263999223709106
Time Elapsed: 10.234907388687134
Accuracy: 0.96063703 Loss: 1.3251297


In [15]:
key = jax.random.PRNGKey(54321)
# Since there's nothing stochastic about the network, we can avoid using an RNG as a param!
SNN = hk.without_apply_rng(hk.transform(nmnist_snn))
params = SNN.init(rng=key, x=x[0])
start = time()
grad_params, metrics = gd(SNN, params, nmnist_dl, epochs=25, schedule=2e-4) 
elapsed = time() - start
print("Performance: train_loss={}, val_acc={}, val_loss={}".format(*metrics[-1]))
print("Time Elapsed:", elapsed)
acc, loss, preds, tgts = test_gd(SNN, grad_params, nmnist_dl)
print("Accuracy:", acc, "Loss:", loss)


  0%|          | 0/25 [00:00<?, ?it/s]

Performance: train_loss=0.9095025062561035, val_acc=0.9585598111152649, val_loss=1.3340524435043335
Time Elapsed: 10.541772603988647
Accuracy: 0.95763224 Loss: 1.3337369
